<a href="https://colab.research.google.com/github/Samir-atra/Other/blob/main/Capsa_experience/Capsa_Copy_of_Train%26Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pathlib
import IPython
import sys
!pip install keras_tuner -q
import keras_tuner
!pip install git+https://github.com/themis-ai/capsa.git
import capsa
from capsa import EnsembleWrapper, DropoutWrapper, VAEWrapper
!pip install helper
import helper

from google.colab import drive             
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/themis-ai/capsa.git to /tmp/pip-req-build-z7ehwiay
  Running command git clone --filter=blob:none --quiet https://github.com/themis-ai/capsa.git /tmp/pip-req-build-z7ehwiay
  Resolved https://github.com/themis-ai/capsa.git to commit d5f0decf7e8d4abdb9ca370a0b9bb03787de8e9a
  Preparing metadata (setup.py) ... done
  Created wheel for capsa: filename=capsa-0.1.5-py3-none-any.whl size=48480 sha256=31d4a4fd5cad9d3b8b343560d8c5658557d83c2320531fa31687549dde0ee629
  Stored in directory: /tmp/pip-ephem-wheel-cache-p6web6i0/wheels/b8/21/2a/cb813978e5d8e8ffb82950eb786e69e6b70288ebea8557ff11
Successfully built capsa
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for helper: filename=

In [ ]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')
data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')

dataset_path = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

efficientnet_dataset = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    seed= 3,
    batch_size=64,
    image_size=(240, 240),
    color_mode="rgb")

# efficientnet_dataset_val = tf.keras.utils.image_dataset_from_directory(
#     data_path,
#     labels= 'inferred',
#     seed= 3,
#     batch_size=64,
#     image_size=(240, 240),
#     color_mode="rgb")

num_classes = 1



Found 4159 files belonging to 2 classes.
Using 3328 files for training.
Found 4159 files belonging to 2 classes.
Using 831 files for validation.
Found 921 files belonging to 2 classes.
Found 4159 files belonging to 2 classes.


In [ ]:
# # catch and prefetch
# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# the HistogramVAEWrapper check


# def build_model(hp):

model = tf.keras.Sequential([
    tf.keras.Input(shape=(64, 64, 3)),
    # tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, strides = 2, padding = "same",  activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),                                                                         #DP1
    tf.keras.layers.Conv2D(32, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),                                                                         #DP2
    tf.keras.layers.Conv2D(64, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, 1, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.7),                                                                         #DP3
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 512, activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP4                                                                        #DP5
    tf.keras.layers.Dense(num_classes, activation= None)
])


decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(units=4*4*128),  
        tf.keras.layers.Reshape((4, 4, 128)),
        tf.keras.layers.Conv2DTranspose(64, 1,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(32, 3,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(16, 3,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(3, 3,  strides=2, padding = "same"),
    ])


# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

wrapped_model = capsa.HistogramVAEWrapper(model, num_bins=5, queue_size=20000, latent_dim = 64, decoder=decoder)

wrapped_model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss= tf.losses.BinaryCrossentropy(from_logits= True),
    metrics= [tf.keras.metrics.BinaryAccuracy()]
)

#   return wrapped_model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="vae_compiled_binary_accuracy",
#     max_trials=5,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Tuner_Capsa",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()

# wrapped_model.summary()

history = wrapped_model.fit(
    dataset_path,
    epochs=30,                                    
    validation_data = dataset_path_val)


out = wrapped_model.predict(dataset_path_test, batch_size=512)


Epoch 1/30


52/52 [==============================] - 804s 12s/step - vae_compiled_loss: 0.8514 - vae_compiled_binary_accuracy: 0.5165 - vae_wrapper_loss: 70154176.0000 - val_vae_compiled_loss: 0.5182 - val_vae_compiled_binary_accuracy: 0.7497 - val_vae_wrapper_loss: 34375212.0000
Epoch 2/30
52/52 [==============================] - 7s 124ms/step - vae_compiled_loss: 0.5485 - vae_compiled_binary_accuracy: 0.7496 - vae_wrapper_loss: 39863024.0000 - val_vae_compiled_loss: 0.4342 - val_vae_compiled_binary_accuracy: 0.8039 - val_vae_wrapper_loss: 32065984.0000
Epoch 3/30
52/52 [==============================] - 7s 123ms/step - vae_compiled_loss: 0.4383 - vae_compiled_binary_accuracy: 0.7955 - vae_wrapper_loss: 29629548.0000 - val_vae_compiled_loss: 0.4008 - val_vae_compiled_binary_accuracy: 0.8171 - val_vae_wrapper_loss: 29190026.0000
Epoch 4/30
52/52 [==============================] - 9s 152ms/step - vae_compiled_loss: 0.4150 - vae_compiled_binary_accuracy: 0.8057 - vae_wrapper_loss: 27020900.0000 - va

In [ ]:
mve_model = capsa.MVEWrapper(model, is_classification=True)

mve_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=[tf.keras.metrics.BinaryAccuracy()], 
    run_eagerly=True
)

history = mve_model.fit(
        dataset_path,
        epochs=6,
        batch_size=64,
  )

normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = dataset_path.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds = dataset_path_test.map(lambda x, y: (normalization_layer(x), y))

images = np.vstack((normalized_test_ds, normalized_train_ds))

# print(type(images[1][0]))

# images= tf.convert_to_tensor(np.asarray(images).astype('float32'))

# predictions = model(images)

Epoch 1/6


52/52 [==============================] - 11s 101ms/step - mve_compiled_loss: 0.4156 - mve_compiled_binary_accuracy: 0.8397 - mve_wrapper_loss: 127.9462
Epoch 2/6
52/52 [==============================] - 6s 96ms/step - mve_compiled_loss: 0.0394 - mve_compiled_binary_accuracy: 0.9920 - mve_wrapper_loss: 128.0146
Epoch 3/6
52/52 [==============================] - 7s 111ms/step - mve_compiled_loss: 0.0076 - mve_compiled_binary_accuracy: 0.9999 - mve_wrapper_loss: 127.5979
Epoch 4/6
52/52 [==============================] - 6s 96ms/step - mve_compiled_loss: 0.0042 - mve_compiled_binary_accuracy: 1.0000 - mve_wrapper_loss: 127.4379
Epoch 5/6
52/52 [==============================] - 7s 118ms/step - mve_compiled_loss: 0.0019 - mve_compiled_binary_accuracy: 1.0000 - mve_wrapper_loss: 127.8286
Epoch 6/6
52/52 [==============================] - 6s 96ms/step - mve_compiled_loss: 0.0013 - mve_compiled_binary_accuracy: 1.0000 - mve_wrapper_loss: 127.5142


In [ ]:
Ensemble_wrapped_model = EnsembleWrapper(model,num_members=1)


Ensemble_wrapped_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
                     )

history = Ensemble_wrapped_model.fit(
        dataset_path,
        epochs=12,
        batch_size=64,
  )

# mesh_grid = helper.get_grid()
# output = Ensemble_wrapped_model(mesh_grid)
# helper.plot_ensemble_classification(output,mesh_grid,dataset_path_test)

Epoch 1/12
52/52 [==============================] - 10s 90ms/step - usermodel_0_compiled_loss: 3.1661 - usermodel_0_compiled_binary_accuracy: 0.7560
Epoch 2/12
52/52 [==============================] - 6s 104ms/step - usermodel_0_compiled_loss: 2.8640 - usermodel_0_compiled_binary_accuracy: 0.8004
Epoch 3/12
52/52 [==============================] - 6s 95ms/step - usermodel_0_compiled_loss: 2.8455 - usermodel_0_compiled_binary_accuracy: 0.8065
Epoch 4/12
52/52 [==============================] - 6s 94ms/step - usermodel_0_compiled_loss: 2.8600 - usermodel_0_compiled_binary_accuracy: 0.8074
Epoch 5/12
52/52 [==============================] - 7s 106ms/step - usermodel_0_compiled_loss: 2.8856 - usermodel_0_compiled_binary_accuracy: 0.8067
Epoch 6/12
52/52 [==============================] - 6s 94ms/step - usermodel_0_compiled_loss: 2.8680 - usermodel_0_compiled_binary_accuracy: 0.8080
Epoch 7/12
52/52 [==============================] - 7s 115ms/step - usermodel_0_compiled_loss: 3.4449 - userm

In [ ]:
base_model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/feature_vector/2", trainable=False),  
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
base_model.build([None, 240, 240, 3])  


dropout_model = DropoutWrapper(base_model, p=0.5)

dropout_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
                     )

history = dropout_model.fit(
        efficientnet_dataset,
        epochs=12,
        batch_size=64,
  )

Epoch 1/12
65/65 [==============================] - 212s 3s/step - dropout_compiled_loss: 0.7917 - dropout_compiled_binary_accuracy: 0.4802 - dropout_wrapper_loss: 0.7380
Epoch 2/12
65/65 [==============================] - 14s 195ms/step - dropout_compiled_loss: 0.6673 - dropout_compiled_binary_accuracy: 0.4845 - dropout_wrapper_loss: 0.6416
Epoch 3/12
65/65 [==============================] - 14s 195ms/step - dropout_compiled_loss: 0.6156 - dropout_compiled_binary_accuracy: 0.4836 - dropout_wrapper_loss: 0.5977
Epoch 4/12
65/65 [==============================] - 14s 197ms/step - dropout_compiled_loss: 0.5749 - dropout_compiled_binary_accuracy: 0.4758 - dropout_wrapper_loss: 0.5707
Epoch 5/12
65/65 [==============================] - 14s 193ms/step - dropout_compiled_loss: 0.5644 - dropout_compiled_binary_accuracy: 0.4821 - dropout_wrapper_loss: 0.5483
Epoch 6/12
65/65 [==============================] - 14s 184ms/step - dropout_compiled_loss: 0.5500 - dropout_compiled_binary_accuracy: 0.

In [ ]:

vae_encoder = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 5, strides = 2, padding = "same",  activation='relu', input_shape=(64, 64, 3)),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    tf.keras.layers.BatchNormalization(),                                                                       #DP3
    tf.keras.layers.Conv2D(32, 5, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 5, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 5, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),                                                                         #DP2
    tf.keras.layers.Conv2D(256, 5, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 1024, activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(1, activation= None)
])

vae_encoder.summary()

vae_decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(units=1*1*64),  
        tf.keras.layers.Reshape((1, 1, 64)),
        tf.keras.layers.Conv2DTranspose(256, 5,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(128, 5,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(64, 5,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(32, 5,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(16, 5,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(3, 3,  strides=2, padding = "same"),
    ])

vae_model = VAEWrapper(vae_encoder, decoder=vae_decoder)

vae_model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0000001),
    loss= tf.losses.BinaryCrossentropy(from_logits= True),
    metrics= [tf.keras.metrics.BinaryAccuracy()]
)

history = vae_model.fit(
    dataset_path,
    epochs=3,                                    
    validation_data = dataset_path_val)

